In [1]:
import os

Constants.

In [2]:
ROOT = '.'
OUT_FOLDER = f'{ROOT}/results'
DATA_FOLDER = f'{ROOT}/data'
FILE = f"{DATA_FOLDER}/Supp. File 1.xlsx"
CHUNKSIZE = 30_000  # Size of chunks for molecular feature calculation
N_JOBS = -1

Skip if modelling files are present.
If you are sure you want to overwrite these files, run the notebook **XX-Advanced_removal_to_run_from_scratch**.

In [3]:
if os.path.exists(f'{DATA_FOLDER}/Papyrus_05-7_BioSpectra_Mordred-ZscalesVanWesten-descriptors-Reg2SMILES-AllSplitsPROPER-DEVIATION-grouped.feather'):
    raise FileExistsError('Files requiring substantial resources are protected. Run the notebook called "XX-Advanced_removal_to_run_from_scratch" to force this notebook to run')
else:
    import pickle
    import re
    from pathlib import Path
    
    import pandas as pd
    from more_itertools import chunked
    from papyrus_scripts import PapyrusDataset
    from sklearn.model_selection import train_test_split
    from pandarallel import pandarallel
    from tqdm.auto import tqdm, trange
    
    from utilities import (ScaffoldSplitter, FrameworkSplitter, generate_scaffold,
                           ConnectedComponentsSplitter,
                           prepare_biospectra_inputs, prepare_biospectra_inputs_from_papyrus, train_bioactivity_spectra_model, make_biospectra_predictions)

Create the bioactivity dataset.

In [ ]:
data = (PapyrusDataset(version='05.7', is3d=False, plusplus=False)
            .keep_quality('medium')
            .to_dataframe(progress=True)
            [['connectivity', 'target_id', 'accession', 'SMILES', 'pchembl_value_Mean']]
            .query('SMILES != "C[C+](=O)(Nc1ccccc1)C(C#N)=Cc1cc(O)c(O)cc1"') # Avoid molecule that RDKit cannot parse
        )

Organize data splitting schemes: scaffold, framework, random and connected components.

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers=8, verbose=0)
data['murcko_scaffold'] = data.SMILES.parallel_apply(generate_scaffold)
data['murcko_cyclic_skeleton'] = data.SMILES.parallel_apply(generate_scaffold, use_csk=True)
# Scaffold split
train_idx, val_idx, test_idx = ScaffoldSplitter()._split(data, scaffold_list=data.murcko_scaffold, random_state=12345678)
data['murcko_scaffold_split'] = (pd.Series(data.index)
                                   .mask(data.index.isin(train_idx), 'train')
                                   .mask(data.index.isin(val_idx), 'val')
                                   .mask(data.index.isin(test_idx), 'test'))
# Framework split
train_idx, val_idx, test_idx = FrameworkSplitter()._split(data, scaffold_list=data.murcko_cyclic_skeleton, random_state=12345678)
data['murcko_cyclic_skeleton_split'] = (pd.Series(data.index)
                                          .mask(data.index.isin(train_idx), 'train')
                                          .mask(data.index.isin(val_idx), 'val')
                                          .mask(data.index.isin(test_idx), 'test'))
# Random split
train_idx, other = train_test_split(data.index, train_size=0.8, random_state=12345678)
val_idx, test_idx = train_test_split(other, train_size=0.5, random_state=12345678)
data['random_split'] = (pd.Series(data.index)
                          .mask(data.index.isin(train_idx), 'train')
                          .mask(data.index.isin(val_idx), 'val')
                          .mask(data.index.isin(test_idx), 'test'))
# Connected components
components = ConnectedComponentsSplitter.get_connected_components(data, 'connectivity', 'accession')
train_idx, val_idx, test_idx = ConnectedComponentsSplitter()._split(components, random_state=12345678, tolerance=int(1e-5))
data['group'] = components.group
data['connected_component_split'] = (pd.Series(data.group)
                                     .mask(data.group.isin(train_idx), 'train')
                                     .mask(data.group.isin(val_idx), 'val')
                                     .mask(data.group.isin(test_idx), 'test'))

Add absolute deviation values of pChEMBL values from the average pChEMBL value of the protein (based on `target_id`). 

In [ ]:
data = pd.concat((data.drop(columns='pchembl_value_Mean'),
                  data.pchembl_value_Mean,
                  (data.pchembl_value_Mean - data.groupby('target_id').pchembl_value_Mean.transform('mean')).rename('pchembl_value_dev')
                  ), axis=1)

Save deviations from average pChEMBL values.

In [8]:
(data.groupby('target_id')
 .pchembl_value_Mean
 .agg('mean')
 .rename('average_pchembl_value_Mean')
 .to_json(f'{OUT_FOLDER}/Protein_average_Papyrus_05-7_BioSpectra_Mordred-ZscalesVanWesten-descriptors-Reg2SMILES-AllSplitsPROPER-DEVIATION.json')
 )

Obtain sequences of the protein targets from the Payrus dataset.

In [5]:
protein_targets = (PapyrusDataset.from_dataframe(data, is3d=False, version='05.7', plusplus=False)
                   .proteins()
                   .to_dataframe())

Drop any compound-protein pair in which proteins have less than 50 amino acids (protein descriptors are not applicable for these).

In [8]:
data = data[data.target_id.isin(protein_targets.query('Sequence.str.len() >= 50').target_id)]

Calculate molecular and protein descriptors altogether. We must use chunks to save memory (since data types cannot be enforced during calculation, only afterwards).

In [9]:
data.to_feather(r'C:\Users\ojbeq\Downloads\save_state_data_for_BS.feather')

In [17]:
data= pd.read_feather(r'C:\Users\ojbeq\Downloads\save_state_data_for_BS.feather')

In [ ]:
inputs = data[['SMILES', 'target_id']].merge(protein_targets[['target_id', 'Sequence']], on='target_id')
# Map each chunk back to a `PapyrusDataset` instance, to allow fast access to pre-calculated descriptors. 
outputs = []
for i in trange(0, len(inputs), CHUNKSIZE):
    output = pd.concat(prepare_biospectra_inputs_from_papyrus(PapyrusDataset.from_dataframe(data.iloc[i: i+CHUNKSIZE],
                                                                                            is3d=False,
                                                                                            version='05.7',
                                                                                            plusplus=False,
                                                                                            chunksize=500_000),
                                                              know_sequences=False,
                                                              data_folder=DATA_FOLDER),
                       axis=0, ignore_index=True)
    id_col = output[['connectivity', 'target_id']]
    output = (output.drop(columns=['connectivity', 'target_id'])
              .apply(pd.to_numeric, downcast='integer')
              .apply(pd.to_numeric, downcast='float'))
    output = pd.concat((id_col, output), axis=1)
    outputs.append(output)
outputs = pd.concat(outputs)

  0%|          | 0/57 [00:00<?, ?it/s]

In [12]:
data = pd.concat((data.reset_index(), outputs), axis=1)

In [15]:
outputs.shape

(1703748, 1764)

In [7]:
data.to_feather(f'{DATA_FOLDER}/Papyrus_05-7_BioSpectra_Mordred-ZscalesVanWesten-descriptors-Reg2SMILES-AllSplitsPROPER-DEVIATION-grouped.feather')

Train bioactivity spectra models.

In [ ]:
for short_split_name, split_type in (('scaffold', 'murcko_scaffold_split'),
                                     ('framework', 'murcko_cyclic_skeleton_split'),
                                     ('random', 'random_split'),
                                     ('doublecold', 'non_overlappling_split')):
    for short_activity_name, activity_type in (('mean', 'pchembl_value_Mean'),
                                               ('dev', 'pchembl_value_dev')):
        os.makedirs(f'{OUT_FOLDER}/{short_split_name}_{short_activity_name}', exist_ok=True)
        train_bioactivity_spectra_model(data,
                                        split_type=split_type,
                                        activity_type=activity_type,
                                        model_type='PCM',
                                        out_folder=f'{DATA_FOLDER}/{short_split_name}_{short_activity_name}')

Determine the model with highest R² to be used for inference.

In [3]:
import pickle
import re
from pathlib import Path

import pandas as pd
from more_itertools import chunked
from tqdm.auto import tqdm

from utilities import (prepare_biospectra_inputs, make_biospectra_predictions)

In [32]:
pattern_holdout = re.compile(r"^Holdout test:\s+MSE Loss: (-?\d+\.\d+)\s+R²: (-?\d+\.\d+)\s+Pearson's r: (-?\d+\.\d+)$")
pattern_best_epoch= re.compile(r"^No loss improvement in the last \d+ epochs: minimum loss = -?\d+\.\d+ from epoch (\d+)$")
pattern_validation= re.compile(r"^\s+Epoch (\d+) \(validation\):\s+MSE Loss: (-?\d+\.\d+)\s+R²: (-?\d+\.\d+)\s+Pearson's r: (-?\d+\.\d+)$")
performances = []
for short_split_name, split_type in (('scaffold', 'murcko_scaffold_split'),
                                     ('framework', 'murcko_cyclic_skeleton_split'),
                                     ('random', 'random_split'),
                                     ('doublecold', 'non_overlappling_split')):
    for short_activity_name, activity_type in (('mean', 'pchembl_value_Mean'),
                                               ('dev', 'pchembl_value_dev')):
        log_file = next(Path(f'{DATA_FOLDER}/{short_split_name}_{short_activity_name}').glob('*.log'))
        with open(log_file, encoding="UTF-8") as fh:
            # Find metrics on validation set
            val_values = (pd.DataFrame([match.groups() for line in fh if (match := pattern_validation.search(line))],
                                       columns=['Best Training Epoch', 'Validation MSE', 'Validation R2', 'Validation Pearson'])
                            .astype({'Best Training Epoch': int, 'Validation MSE': float, 'Validation R2': float, 'Validation Pearson': float}))
            fh.seek(0)
            # Find the epoch with the best validation loss
            best_epoch = int([match.groups() for line in fh if (match := pattern_best_epoch.search(line))][0][0])
            fh.seek(0)
            # Find metrics on holdout set
            holdout_values = (pd.DataFrame([match.groups() for line in fh if (match := pattern_holdout.search(line))],
                                           columns=['Test MSE', 'Test R2', 'Test Pearson'])
                              .astype({'Test MSE': float, 'Test R2': float, 'Test Pearson': float}))
            # Aggregate
            performances.append(pd.concat((pd.DataFrame([{'data splitting scheme': short_split_name, 'pChEMBL value': short_activity_name}]),
                                           val_values.query('`Best Training Epoch` == @best_epoch').reset_index(drop=True),
                                           holdout_values), axis=1))

performances = pd.concat(performances, ignore_index=True)
performances

,data splitting scheme,pChEMBL value,Best Training Epoch,Validation MSE,Validation R2,Validation Pearson,Test MSE,Test R2,Test Pearson
0,scaffold,mean,2474,0.464,0.752,0.868,0.473,0.749,0.866
1,scaffold,dev,2612,0.445,0.579,0.762,0.457,0.587,0.767
2,framework,mean,2231,0.470,0.745,0.864,0.487,0.734,0.857
3,framework,dev,1636,0.449,0.573,0.758,0.468,0.556,0.747
4,random,mean,3119,0.353,0.805,0.900,0.347,0.808,0.900
5,random,dev,1831,0.347,0.668,0.818,0.347,0.669,0.818
6,doublecold,mean,9,2.001,0.148,0.509,1.917,-0.334,0.220
7,doublecold,dev,851,0.934,-0.244,0.126,0.790,-1.019,0.023


The random split model predicting mean values will be used for bioactivity spectrum prediction as it shows the best performance and no extrapolation capacity to new proteins will be performed (the proteins making up the spetrum will remain constant).

In [33]:
model_prefix = next(Path(f'{DATA_FOLDER}/random_mean').glob('*.pkg')).absolute().as_posix().replace('.pkg', '')

In [34]:
data_mols = pd.read_excel(FILE, sheet_name='Original Dataset MolDescs',
                          usecols=['InChIKey', 'SMILES', 'ALogP', 'Molecular_Weight', 'Molecular_Solubility', 'H_Count', 'C_Count',
                                   'N_Count', 'O_Count', 'F_Count', 'S_Count', 'Cl_Count', 'Num_H_Acceptors_Lipinski', 'Num_H_Donors_Lipinski', 'JY', 'Wiener',
                                   'CHI_V_3_P', 'CHI_V_3_C', 'ES_Sum_sCH3', 'ES_Sum_ssCH2', 'ES_Sum_dsCH', 'ES_Sum_aaCH', 'ES_Sum_sssCH', 'ES_Sum_dssC', 'ES_Sum_aasC',
                                   'ES_Sum_aaaC', 'ES_Sum_ssssC', 'ES_Sum_sNH2', 'ES_Sum_ssNH', 'ES_Sum_aaN', 'ES_Sum_sssN', 'ES_Sum_ddsN', 'ES_Sum_sOH', 'ES_Sum_dO',
                                   'ES_Sum_ssO', 'ES_Sum_ssS', 'Kappa_3_AM', 'PHI', 'ECFP_6'])
comp_mols = pd.read_excel(FILE, sheet_name='Additional Set MolDescs',
                          usecols=['SMILES', 'binaryDILI', 'vDILIConcern', 'ALogP', 'Molecular_Weight', 'Molecular_Solubility', 'H_Count', 'C_Count',
                                   'N_Count', 'O_Count', 'F_Count', 'S_Count', 'Cl_Count', 'Num_H_Acceptors_Lipinski', 'Num_H_Donors_Lipinski', 'JY', 'Wiener',
                                   'CHI_V_3_P', 'CHI_V_3_C', 'ES_Sum_sCH3', 'ES_Sum_ssCH2', 'ES_Sum_dsCH', 'ES_Sum_aaCH', 'ES_Sum_sssCH', 'ES_Sum_dssC', 'ES_Sum_aasC',
                                   'ES_Sum_aaaC', 'ES_Sum_ssssC', 'ES_Sum_sNH2', 'ES_Sum_ssNH', 'ES_Sum_aaN', 'ES_Sum_sssN', 'ES_Sum_ddsN', 'ES_Sum_sOH', 'ES_Sum_dO',
                                   'ES_Sum_ssO', 'ES_Sum_ssS', 'Kappa_3_AM', 'PHI', 'ECFP_6'])

ECFP6 = (data_mols['ECFP_6']
             .apply(list)
             .apply(pd.Series)
             .astype(int)
             .rename_axis(index=None, columns=None)
             .rename(columns=lambda x: f'ECFP_6_{x+1}'))

data_mols = pd.concat((data_mols.drop(columns=['ECFP_6']), ECFP6),
                      axis=1)

ECFP6 = (comp_mols['ECFP_6']
             .apply(list)
             .apply(pd.Series)
             .astype(int)
             .rename_axis(index=None, columns=None)
             .rename(columns=lambda x: f'ECFP_6_{x+1}'))

comp_mols = pd.concat((comp_mols.drop(columns=['ECFP_6']), ECFP6),
                      axis=1)

In [36]:
predictions = []
for chunk in tqdm(chunked(data_mols.SMILES.drop_duplicates(),n=10), total=-(-len(data_mols) // 10)):
    inputs = prepare_biospectra_inputs(smiles=chunk, know_sequences=True, data_folder=DATA_FOLDER, verbose=False)
    predictions.append(pd.concat((inputs[['SMILES', 'target_id']],
                              make_biospectra_predictions(inputs.drop(columns=['SMILES', 'target_id']),
                                                          model_prefix=model_prefix,
                                                          verbose=False)), axis=1)
                         .pivot(index='SMILES', columns='target_id')
                         .reset_index())

predictions = pd.concat(predictions, ignore_index=True)
predictions.columns = [col[0] if i == 0 else col[1] for i, col in enumerate(predictions.columns)]
data_chem_biospectra = data_mols.merge(predictions, on='SMILES').drop(columns='SMILES')

  0%|          | 0/12 [00:00<?, ?it/s]

In [37]:
predictions = []
for chunk in tqdm(chunked(comp_mols.SMILES.drop_duplicates(),n=10), total=-(-len(comp_mols) // 10)):
    inputs = prepare_biospectra_inputs(smiles=chunk, know_sequences=True, data_folder=DATA_FOLDER, verbose=False)
    predictions.append(pd.concat((inputs[['SMILES', 'target_id']],
                              make_biospectra_predictions(inputs.drop(columns=['SMILES', 'target_id']),
                                                          model_prefix=model_prefix,
                                                          verbose=False)), axis=1)
                         .pivot(index='SMILES', columns='target_id')
                         .reset_index())

predictions = pd.concat(predictions, ignore_index=True)
predictions.columns = [col[0] if i == 0 else col[1] for i, col in enumerate(predictions.columns)]
comp_data_biospectra = comp_mols.merge(predictions, on='SMILES').drop(columns='SMILES')

  0%|          | 0/25 [00:00<?, ?it/s]

Save to disk.
These values are those in the last tab of `Supp. File. 1.xlsx` 

In [38]:
with open(f'{OUT_FOLDER}/data_chem_biospectra_from_scratch.pkl', 'wb') as oh:
    pickle.dump(data_chem_biospectra, oh)
with open(f'{OUT_FOLDER}/comp_data_biospectra_from_scratch.pkl', 'wb') as oh:
    pickle.dump(comp_data_biospectra, oh)